In [48]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

In [50]:
data = pd.read_csv('../Dataset/combined_final_dataset.csv')
data["main_category"] = LabelEncoder().fit_transform(data["main_category"])
z = data.pop("date")
m = data.pop("category")
n = data.pop("Title")
y = data.pop('main_category')

In [51]:
print(data.head())

                                         description
0  health expert said early predict whether deman...
1  subdued passenger crew fled back aircraft conf...
2                         dog understand could eaten
3  accidentally put toothpaste toothbrush screame...
4  amy cooper accused investment firm franklin te...


In [52]:
import numpy
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

#convert to numpy array

y_train = numpy.array(y_train)
y_test = numpy.array(y_test)

In [53]:
vocab_size = 30000
embedding_dim = 32
max_length = 300
trunc_type='post'
padding_type='post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

2


In [55]:
sequences = tokenizer.texts_to_sequences(x_train)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
#



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1000, 300)         270000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 256)               439296    
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 710324 (2.71 MB)
Trainable params: 710324 (2.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
from sklearn.utils import class_weight
import numpy as np

# Assuming 'y_train' contains your integer-encoded class labels
class_labels = np.unique(y_train)

# Calculate class weights
class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = class_labels, y = y_train)

# Create a dictionary mapping class indices to their respective weights
class_weight_dict = dict(zip(class_labels, class_weights))

print("Class Weights:", class_weight_dict)

Class Weights: {0: 5.675925925925926, 1: 0.5434397163120568, 2: 0.5916988416988417, 3: 3.4055555555555554}
